<a href="https://colab.research.google.com/github/csikasote/xls-r-bem-exp/blob/main/xls_r_bemba_fusing_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets torchaudio
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

     |████████████████████████████████| 311 kB 4.3 MB/s 
     |████████████████████████████████| 3.5 MB 64.9 MB/s 
     |████████████████████████████████| 133 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 61.1 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 243 kB 70.2 MB/s 
     |████████████████████████████████| 6.8 MB 41.2 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 271 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 54.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-89h9pnpy
  Running command git clone -q https://github.com/huggingface/transform

In [3]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset, load_metric
from tqdm import tqdm

import os, string, six, re, sys
import IPython.display as ipd
import pandas as pd
import numpy as np
import torchaudio
import torch
import argparse
from typing import Dict

from transformers import AutoFeatureExtractor, pipeline

In [12]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [13]:
!git config --global credential.helper store

In [14]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
Th

In [15]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

--2022-02-18 07:45:36--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K   522KB/s    in 0.9s    

2022-02-18 07:45:37 (522 KB/s) - written to stdout [491888/491888]

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile

In [16]:
from datasets import load_dataset

username = "csikasote"  # change to your username

dataset = load_dataset("csikasote/bembaspeech_plus_jw_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))

Downloading:   0%|          | 0.00/676 [00:00<?, ?B/s]

Using custom data configuration csikasote--bembaspeech_plus_jw_processed-a2de8476db2b0708


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/csikasote--bembaspeech_plus_jw_processed-a2de8476db2b0708/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [17]:
!kenlm/build/bin/lmplz -o 5  <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1918697472 bytes == 0x55b150e80000 @  0x7f201eac11e7 0x55b14e8117e2 0x55b14e7ac4fe 0x55b14e78b2eb 0x55b14e777066 0x7f201cc5abf7 0x55b14e778baa
tcmalloc: large alloc 8953896960 bytes == 0x55b1c3450000 @  0x7f201eac11e7 0x55b14e8117e2 0x55b14e80080a 0x55b14e801248 0x55b14e78b308 0x55b14e777066 0x7f201cc5abf7 0x55b14e778baa
****************************************************************************************************
Unigram tokens 4736180 types 174714
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:2096568 2:1062990784 3:1993107840 4:3188972288 5:4650585088
tcmalloc: large alloc 4650590208 bytes == 0x55b150e80000 @  0x7f201eac11e7 0x55b14e8117e2 0x55b14e80080a 0x55b14e801248 0x55b14e78b8d7 0x55b14e777066 0x7f201cc5abf7 0x55b14e778baa
tcmalloc: large alloc 1993113600 bytes == 0

In [19]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [21]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("csikasote/wav2vec2-large-xls-r-1b-bemba-fds-st-v1")

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/260 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/259 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [22]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

In [24]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="/content/5gram_correct.arpa",
)

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


In [25]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

In [26]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [27]:
from huggingface_hub import Repository

repo = Repository(local_dir="wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm", clone_from="csikasote/wav2vec2-large-xls-r-1b-bemba-fds-st-v1")

Cloning https://huggingface.co/csikasote/wav2vec2-large-xls-r-1b-bemba-fds-st-v1 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.47k/3.59G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.05k/3.05k [00:00<?, ?B/s]

Clean file training_args.bin:  33%|###2      | 1.00k/3.05k [00:00<?, ?B/s]

Download file runs/Feb17_23-07-14_79e97fcd3b25/1645139645.0127013/events.out.tfevents.1645139645.79e97fcd3b25.…

Clean file runs/Feb17_23-07-14_79e97fcd3b25/1645139645.0127013/events.out.tfevents.1645139645.79e97fcd3b25.136…

Download file runs/Feb17_23-07-14_79e97fcd3b25/events.out.tfevents.1645139644.79e97fcd3b25.1366.0:  11%|#1    …

Clean file runs/Feb17_23-07-14_79e97fcd3b25/events.out.tfevents.1645139644.79e97fcd3b25.1366.0:   4%|3        …

Clean file pytorch_model.bin:   0%|          | 1.00k/3.59G [00:00<?, ?B/s]

In [28]:
processor_with_lm.save_pretrained("wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm")

In [29]:
!tree -h wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/

wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/
├── [  23]  added_tokens.json
├── [ 193]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [570M]  5gram_correct.arpa
│   ├── [  78]  attrs.json
│   └── [1.8M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [3.6G]  pytorch_model.bin
├── [4.2K]  README.md
├── [4.0K]  runs
│   └── [4.0K]  Feb17_23-07-14_79e97fcd3b25
│       ├── [4.0K]  1645139645.0127013
│       │   └── [4.7K]  events.out.tfevents.1645139645.79e97fcd3b25.1366.1
│       └── [ 26K]  events.out.tfevents.1645139644.79e97fcd3b25.1366.0
├── [ 502]  special_tokens_map.json
├── [ 353]  tokenizer_config.json
├── [3.0K]  training_args.bin
└── [ 259]  vocab.json

4 directories, 15 files


In [30]:
!kenlm/build/bin/build_binary /content/wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/language_model/5gram_correct.arpa /content/wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/language_model/5gram.bin

Reading /content/wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [31]:
!rm /content/wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/language_model/5gram_correct.arpa && tree -h wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/

wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/
├── [  23]  added_tokens.json
├── [ 193]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [273M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.8M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [3.6G]  pytorch_model.bin
├── [4.2K]  README.md
├── [4.0K]  runs
│   └── [4.0K]  Feb17_23-07-14_79e97fcd3b25
│       ├── [4.0K]  1645139645.0127013
│       │   └── [4.7K]  events.out.tfevents.1645139645.79e97fcd3b25.1366.1
│       └── [ 26K]  events.out.tfevents.1645139644.79e97fcd3b25.1366.0
├── [ 502]  special_tokens_map.json
├── [ 353]  tokenizer_config.json
├── [3.0K]  training_args.bin
└── [ 259]  vocab.json

4 directories, 15 files


In [32]:
!tree -h wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/

wav2vec2-large-xls-r-1b-bemba-fds-st-v1-with-lm/
├── [  23]  added_tokens.json
├── [ 193]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [273M]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.8M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [3.6G]  pytorch_model.bin
├── [4.2K]  README.md
├── [4.0K]  runs
│   └── [4.0K]  Feb17_23-07-14_79e97fcd3b25
│       ├── [4.0K]  1645139645.0127013
│       │   └── [4.7K]  events.out.tfevents.1645139645.79e97fcd3b25.1366.1
│       └── [ 26K]  events.out.tfevents.1645139644.79e97fcd3b25.1366.0
├── [ 502]  special_tokens_map.json
├── [ 353]  tokenizer_config.json
├── [3.0K]  training_args.bin
└── [ 259]  vocab.json

4 directories, 15 files


In [33]:
repo.push_to_hub(commit_message="Updated lm-boosted decoder")

Upload file language_model/5gram.bin:   0%|          | 3.36k/273M [00:00<?, ?B/s]

To https://huggingface.co/csikasote/wav2vec2-large-xls-r-1b-bemba-fds-st-v1
   924c1ea..c57ac6e  main -> main



'https://huggingface.co/csikasote/wav2vec2-large-xls-r-1b-bemba-fds-st-v1/commit/c57ac6e3240498decbd1106df9388b21fe9fbb66'